In [79]:
#r "../../bin/Debug/net9.0/DiscordTaskBotV2.dll"
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite"
#r "nuget: Microsoft.Data.Sqlite"
#r "nuget: SQLitePCLRaw.bundle_e_sqlite3"

using Microsoft.EntityFrameworkCore.Sqlite;
using Microsoft.EntityFrameworkCore;
using Microsoft.Data.Sqlite;
using DiscordTaskBot.Core;
using DiscordTaskBot.Infrastructure;
using SQLitePCL;

Installed Packages Microsoft.Data.Sqlite, 9.0.8 Microsoft.EntityFrameworkCore.Sqlite, 9.0.8 SQLitePCLRaw.bundle_e_sqlite3, 3.0.1

In [80]:
const ulong TASK_CHANNEL = 123456789;

const ulong ADMIN_ID = 10000;
const ulong USER_ID = 12345;


var task1 = new TaskItem("do something", new TaskDuration(DateTime.UtcNow, DateTime.UtcNow.AddDays(10), TimeZoneInfo.Utc), USER_ID); // Task for USER, 10 days to complete
task1.SetLocation(new TaskLocation(TASK_CHANNEL, 1234));

var task2 = new TaskItem("do something else", new TaskDuration(DateTime.UtcNow, DateTime.UtcNow.AddDays(5), TimeZoneInfo.Utc), ADMIN_ID);
task2.SetLocation(new TaskLocation(TASK_CHANNEL, 2345)); // Task for ADMIN, 5 days to complete

In [81]:
SQLitePCL.Batteries_V2.Init();

var connection = new SqliteConnection("DataSource=:memory:");
connection.Open();

var options = new DbContextOptionsBuilder<TaskItemDbContext>()
    .UseSqlite(connection)
    .Options;

var dbContext = new TaskItemDbContext(options);
ITaskRepository taskRepository = new TaskRepository(dbContext);

async Task WriteAllTasks() {
    foreach (var task in await taskRepository.GetAllTasksAsync()) {
        Console.WriteLine("result:" + task.Description);
    }
}

In [82]:
// AddTaskAsync and GetAllTasksAsync Test
await WriteAllTasks();

await taskRepository.AddTaskAsync(task1);

await WriteAllTasks();

result:do something


In [83]:
Console.WriteLine("result:" + await taskRepository.DeleteTaskByIDAsync(task1.ID));

await WriteAllTasks();

Console.WriteLine("result:" + await taskRepository.DeleteTaskByIDAsync("non existent id"));

result:True
result:False


In [84]:
await taskRepository.AddTaskAsync(task2);

Console.WriteLine("result:" + await taskRepository.GetTaskByIDAsync(task2.ID));

Console.WriteLine("result:" + await taskRepository.GetTaskByIDAsync(task1.ID));

result:DiscordTaskBot.Core.TaskItem
result:


In [85]:
Console.WriteLine("result:" + (await taskRepository.GetTaskByIDAsync(task2.ID)).State);

task2.ChangeState(TaskState.IN_PROGRESS);

await taskRepository.UpdateTaskAsync(task2);

Console.WriteLine("result:" + (await taskRepository.GetTaskByIDAsync(task2.ID)).State);

result:NOT_STARTED
result:IN_PROGRESS


In [86]:
Console.WriteLine("result:" + (await taskRepository.GetTaskByIDAsync(task2.ID)).State);
try {
    task2.ChangeState(TaskState.ARCHIVED);
}
catch (Exception ex) {
    Console.WriteLine(ex.Message);
}

await taskRepository.UpdateTaskAsync(task2);

Console.WriteLine("result:" + (await taskRepository.GetTaskByIDAsync(task2.ID)).State);

result:IN_PROGRESS
Cannot change state from IN_PROGRESS to ARCHIVED
result:IN_PROGRESS
